In [5]:
from datasets import load_dataset
import tiktoken
from multiprocessing import Pool
import numpy as np
import os
import tqdm

c:\Users\joshu\BrawlhallaReplays\.bin\archive\repos\BundtCake\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("Stegvean/CakeGenomes")
dataset

c:\Users\joshu\BrawlhallaReplays\.bin\archive\repos\BundtCake\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joshu\.cache\huggingface\hub\datasets--Stegvean--CakeGenomes. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 214839 examples [01:17, 2754.78 examples/s]
Generating

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 214839
    })
    test: Dataset({
        features: ['text'],
        num_rows: 86848
    })
})

In [3]:
dataset

NameError: name 'dataset' is not defined

In [ ]:
# split dataset
split_dataset = dataset["train"].train_test_split(test_size=0.1, shuffle=True)
split_dataset["val"] = split_dataset.pop('test') 
[split_dataset["train"][i] for i in range(5)]

### tokenizer

In [ ]:
from foo import extract_unique

# Process dataset in parallel using multiprocessing
num_proc = 18  # Adjust based on your CPU cores
with Pool(num_proc) as pool:
    results = pool.map(extract_unique, dataset["train"].iter(batch_size=1000))

# Combine results to get the final unique set
unique_gene_ids = set().union(*results)

print(unique_gene_ids)
vocab_size = len(unique_gene_ids)
print(vocab_size)

NameError: name 'Pool' is not defined

In [ ]:
# export tokenizer data
import json
json.dump(
    {"id_vocab": unique_gene_ids, "vocab_size": vocab_size}, 
    open("tokenizer_data.json"))

In [4]:
id_to_int = { id:i for i,id in enumerate(unique_gene_ids)}
int_to_id = { i:id for i,id in enumerate(unique_gene_ids)}
encode = lambda id_list: [id_to_int[id] for id in id_list]
decode = lambda int_list: [int_to_id[i] for i in int_list]

process = lambda id_str: encode(id_str.split(" "))
print(encode("WP_#####.1"))

NameError: name 'unique_gene_ids' is not defined

In [ ]:
tokenized = split_dataset.map(
    process,
    remove_columns=['text'],
    desc="tokenizing the splits",
    num_proc=num_proc,
)

In [ ]:
# concatenate all the ids in each dataset into one large file we can use for training
for split, dset in tokenized.items():
    arr_len = np.sum(dset['len'], dtype=np.uint64)
    filename = os.path.join(os.path.dirname(__file__), f'{split}.bin')
    dtype = np.uint16 # (can do since enc.max_token_value == 50256 is < 2**16)
    arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
    total_batches = 1024

    idx = 0
    for batch_idx in tqdm(range(total_batches), desc=f'writing {filename}'):
        # Batch together samples for faster write
        batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
        arr_batch = np.concatenate(batch['ids'])
        # Write into mmap
        arr[idx : idx + len(arr_batch)] = arr_batch
        idx += len(arr_batch)
    arr.flush()
